# Spectral Trust Demo

This notebook demonstrates how to use the `spectral_trust` package to analyze a simple sentence.

## ⚠️ **IMPORTANT: RESTART KERNEL** ⚠️
If you are seeing errors, please go to **Kernel > Restart & Clear Output** before running this notebook. This ensures the correct local code is loaded.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path

# Force usage of local src version to get latest fixes
current_dir = Path(os.getcwd())
src_path = current_dir.parent / "src"

if src_path.exists():
    # Insert at position 0 to take precedence over installed packages
    if str(src_path) not in sys.path:
        sys.path.insert(0, str(src_path))
    print(f"Added local source to path: {src_path}")
else:
    print(f"Warning: Local source path not found at {src_path}")


In [2]:
import spectral_trust
print(f"Loaded spectral_trust from: {spectral_trust.__file__}")

# Verify we are loading the local version
if "site-packages" in str(spectral_trust.__file__):
    print("\n⚠️ WARNING: You are loading the installed package instead of the local code!")
    print("Please RESTART THE KERNEL to clear the cached imports.")
else:
    print("✅ Success: Loading local development version.")

In [4]:
from spectral_trust import GSPDiagnosticsFramework, GSPConfig
import torch

# Check if CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [5]:
# Initialize configuration
model_name = "meta-llama/Llama-3.2-1B"  # Full HuggingFace Model ID
config = GSPConfig(
    model_name=model_name,
    device=device,
    save_plots=True,
    output_dir="./demo_results",
    model_kwargs={
        "attn_implementation": "eager",
        "output_attentions": True,      # Requests attention weights
        "output_hidden_states": True    # Requests hidden states
    }
)

In [6]:
# Run analysis
text = "The capital of France is Paris."

with GSPDiagnosticsFramework(config) as framework:
    framework.instrumenter.load_model(config.model_name)
    results = framework.analyze_text(text)
    
    print(f"Analyzed: {text}")
    print("\nFirst 5 layers energy:")
    for diag in results['layer_diagnostics'][:5]:
        print(f"Layer {diag.layer}: {diag.energy:.4f}")